In [3]:
import sys
sys.path.insert(0, '/Users/gulugulu/repos/PuningAnalysis/src')

In [4]:
from simulation.waiting_time_simulator import generate_ancestor, average_substitution, get_descrip_stat, simulate_seq

from maths.evolutionary_rate import calculate_stationary_distribution, calculate_stationary_distribution, calculate_non_statioanry_mu
from cogent3.maths.matrix_exponential_integration import expected_number_subs
import numpy as np
import plotly.express as px

In [5]:
#parameter set up
import json
with open('../../../data/dataset1_16s-10_initial_alignments/matrix_motif.json', 'r') as file:
    matrix_motif = json.load(file)


In [6]:
Q = {'0': np.array(matrix_motif['0']['matrix']['200580'])}
pi_0 = matrix_motif['0']['motif_prob']
pi_stationary = calculate_stationary_distribution(Q['0'])
markov_order = 0
t = 0.5

In [7]:
repeat_times = [100, 200, 500]

In [8]:
length_list = [50, 500, 5000]
ances_seq = [generate_ancestor(n, pi_0) for n in length_list]

In [9]:
#theoretical value of ENS
theorectial_ns = expected_number_subs(pi_0, Q['0'], 0.5)

In [10]:
theorectial_ns

0.39739721133458367

In [11]:
# simulated_ns_dict_100repeats = {}
# for i in range(len(length_list)):
#     print(length_list[i])
#     for r in repeat_times:
#         print(r)
#         simulated_ns_dict_100repeats[(length_list[i], r)] = average_substitution(Q, t, ances_seq[i], r, markov_order)


In [12]:
# simulated_ns_dict_converted = {str(list(key)): value for key, value in simulated_ns_dict_100repeats.items()}

In [13]:
# with open('../../../results/seq_len_test.json', 'w') as outfile:
#     json.dump(simulated_ns_dict_converted, outfile, indent=4)

In [14]:
with open('../../../results/seq_len_test.json', 'r') as infile:
    simulated_ns_dict_seq_length = json.load(infile)

In [17]:
descrip_stat_length = get_descrip_stat(simulated_ns_dict_seq_length)

In [18]:
# fig = px.histogram(x=ns_list, nbins=10)

# fig.update_layout(
#     title_text='Number Distribution',
#     xaxis_title_text='Number of Substitutions',
#     yaxis_title_text='Count',
#     bargap=0.1,
# )

# # Add a vertical line for the average
# fig.add_vline(x=average, line_width=3, line_dash="dash", line_color="red")
# fig.add_vline(x=a, line_width=3, line_dash="dash", line_color="lightgreen")
# fig.update_xaxes(range=[0.3, 0.5])
# fig.update_yaxes(range=[0, 40])

# fig.show()

In [19]:

sub_descrip_stat_length = [descrip_stat_length['[50, 100]'], descrip_stat_length['[500, 100]'], descrip_stat_length['[5000, 100]']]
lengths = [50, 500, 5000]
cvs = []

for val in sub_descrip_stat_length:
    cvs.append(val['coefficient_of_variation'])

# Create the plot
fig = px.scatter(x=lengths, y=cvs, labels={'x': 'Sequence Length', 'y': 'Coefficient of Variation'},
                 title='Coefficient of Variation vs.Sequence Length')

# Adding line for better trend visualization
fig.add_scatter(x=lengths, y=cvs, mode='lines', showlegend=False)

# Show the plot
fig.show()

In [20]:
from scipy import stats
t_test_stat = {}
data = [simulated_ns_dict_seq_length['[50, 100]'][0], simulated_ns_dict_seq_length['[500, 100]'][0], simulated_ns_dict_seq_length['[5000, 100]'][0]]

for i in range(3):
    time = lengths[i]
    sample_values = data[i]
    theoretical_mean = theorectial_ns
    t_stat, p_value = stats.ttest_1samp(sample_values, theoretical_mean)
    t_test_stat[time] = [t_stat, p_value]

In [21]:
t_test_stat

{50: [9.427241652336727, 1.943456991770361e-15],
 500: [-8.600127619694504, 1.2225728282612727e-13],
 5000: [-3.602595013196057, 0.0004948746272951681]}

In [25]:
Q_ = {'0': np.array(matrix_motif['0']['matrix']['200580'])}
pi_0 = matrix_motif['0']['motif_prob']
repeats = 50
markov_order = 0
length = 4000
time_range = [0.5, 1, 2, 3]
ances_seq = generate_ancestor(length, pi_0)
# Dictionary to store results
results = {}

# Loop over each sequence length and time to perform simulations
for t in time_range:
    print(t)
    length = 4000
    ns_per_site_list, avg_ns_per_site = average_substitution(Q, t, ances_seq, repeats, markov_order)
    # Store the results
    results[f"Length {length}, Time {t}"] = {
        'avg_ns_per_site': avg_ns_per_site,
        'ns_per_site_list': ns_per_site_list
    }

with open('../../../results/length_4000.json', 'w') as outfile:
    json.dump(results, outfile, indent=4)

0.5
1
2
3
